In [77]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display

In [19]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [47]:
import pickle

In [9]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [30]:
df=pd.read_csv('bank-additional-full.csv',sep=';')

In [31]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [32]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

In [33]:
categorical_columns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome',
 'y']

In [34]:
categorical=['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [35]:
df.y = (df.y == 'yes').astype(int)

In [36]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [39]:
numerical=['age','duration','campaign','pdays','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']
categorical=['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [40]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [15]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [16]:
C = 1.0
n_splits = 5

In [41]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.y.values
    y_val = df_val.y.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)


In [43]:
print('%.3f +- %.3f'%(np.mean(scores),np.std(scores)))

0.932 +- 0.003


In [46]:
dv, model = train(df_full_train, df_full_train.y.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.y.values
auc = roc_auc_score(y_test, y_pred)
auc

0.9316332090922189

In [50]:
output_file=f'model_C={C}.bin'

In [51]:
f_out=open(output_file,'wb') #write, binary
pickle.dump((dv,model),f_out)
f_out.close()

## Load the model

In [52]:
with open(output_file,'rb') as f_in:
    dv,model=pickle.load(f_in)
    #auto closed the file

In [53]:
dv,model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [78]:
np.set_printoptions(threshold=sys.maxsize)

In [83]:
set(df.marital.values)

{'divorced', 'married', 'single', 'unknown'}

In [84]:
customer={'age': 27,
 'job': 'student',
 'marital': 'single',
 'education': 'university.degree',
 'default': 'yes',
 'housing': 'no',
 'loan': 'no',
 'contact': 'cellular',
 'month': 'may',
 'day_of_week': 'mon',
 'duration': 55,
 'campaign': 3,
 'pdays': 999,
 'previous': 0,
 'poutcome': 'nonexistent',
 'emp.var.rate': 1.1,
 'cons.price.idx': 93.994,
 'cons.conf.idx': -36.4,
 'euribor3m': 4.857,
 'nr.employed': 5191.0,
 'y': 0}

In [85]:
X=dv.transform([customer])

In [86]:
model.predict_proba(X)[0,1]

0.006569082116736781